In [40]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

In [41]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-05-27 08:34:00--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   146MB/s    in 0.2s    

2024-05-27 08:34:00 (146 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Users.csv            


In [42]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [43]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [44]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271379 non-null  object
 1   title   271379 non-null  object
 2   author  271377 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [45]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [46]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 17.5+ MB


In [47]:
df_ratings.user.unique()
print(len(df_ratings.user.unique()))

105283


In [48]:
matrix_df = df_ratings

user_counts = matrix_df['user'].value_counts()
isbn_counts = matrix_df['isbn'].value_counts()

# Create masks
user_mask = matrix_df['user'].isin(user_counts[user_counts >= 200].index)
isbn_mask = matrix_df['isbn'].isin(isbn_counts[isbn_counts >= 100].index)

df_unique = pd.merge(right=matrix_df[user_mask & isbn_mask],
                     left=df_books, on="isbn").drop_duplicates(["title", "user"])

# Create pivot table with values used for training matrix
pivot = df_unique.pivot(index='title', columns='user', values='rating').fillna(0)
matrix = pivot.values
print(matrix.shape)

(673, 888)


In [74]:
# function to return recommended books - this will be tested
def get_recommends(book="", k=5):
    if book not in pivot.index:
        return "Book not found in the dataset."

    # Normalize the data
    scaler = MinMaxScaler()
    normalized_matrix = scaler.fit_transform(matrix)

    # Fit KNN model
    knn = NearestNeighbors(n_neighbors=k+1, metric='cosine', algorithm='brute')
    knn.fit(normalized_matrix)

    # Find the index of the book
    book_index = pivot.index.get_loc(book)
    book_vector = normalized_matrix[book_index].reshape(1, -1)

    # Find k+1 nearest neighbors
    distances, indices = knn.kneighbors(book_vector, n_neighbors=k+1)

    # Collect recommended books, excluding the input book
    recommended_books = []
    for distance, index in zip(distances[0], indices[0]):
        if pivot.index[index] != book:
            recommended_books.append([pivot.index[index], distance])

    return [book, recommended_books[::-1]]

get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['The Gunslinger (The Dark Tower, Book 1)', 0.7579582],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.7519877],
  ['Interview with the Vampire', 0.71920276],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5192908],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.49423724]]]

In [76]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.80807424], ['I Know This Much Is True', 0.77921486], ['The Weight of Water', 0.77896875], ['The Surgeon', 0.7677419], ['The Lovely Bones: A Novel', 0.7282765]]]


In [77]:
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

You passed the challenge! 🎉🎉🎉🎉🎉
